<a href="https://colab.research.google.com/github/Pandey0809/jetBrains-academy/blob/main/multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import cvxpy as cp

In [2]:
h1=np.array([[1,0],[1,0],[1,0]])
h2=np.array([[0,1],[0,1],[0,1]])
h3=np.array([[0,1],[0,1],[0,1]])
h4=np.array([[1,0],[1,0],[1,0]])
y=np.array([[1,0],[1,0],[1,0]])
y_w=np.array([[0,1],[0,1],[0,1]])

In [3]:
w=cp.Variable(shape=(4,1),nonneg=True)
majority=w[0]@h1+w[1]@h2+w[2]@h3+w[3]@h4

In [4]:
majority

Expression(AFFINE, NONNEGATIVE, (3, 2))

In [5]:
#mu_1=0
#mu_2=0
#for i,j,k in zip(majority,y,y_w):
#  mu_1=mu_1+(i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1]))
#  mu_2=mu_2+((i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1])))**2



In [6]:
mu_1=mu_1/4
mu_2=mu_2/4

NameError: ignored

In [ ]:
t=.25
value=0
actual_u=0
u1=np.linspace(0,.75,10)
for u in u1: 
  fn=u**2/mu_2
  problem = cp.Problem(cp.Maximize(fn),[mu_1==u,cp.sum(w)==1,w>=0,fn>=0])
  op=problem.solve()
  print(u,w.value)

In [7]:
actual_u

NameError: ignored

For 4 different trees


In [25]:
# random  splits of data for training 
# random feature selection 
# train
# remember random features 
# predict on known features for x test
#append the prediction on the dictionary
import pandas as pd

In [26]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

In [27]:
#import data
X = breast_cancer.data
Y = breast_cancer.target
#X = X[0:5, :]
#Y = Y[0:5]
data = pd.DataFrame(X, columns=breast_cancer.feature_names)   # converting dataset into pandas dataframe for preprocessing
data['class'] = Y


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('class',axis=1),data.iloc[:,30:31], test_size=0.3, random_state=42)

In [70]:
n=4
H=[]
for i in range(0,n):
  features=data.drop('class',axis=1).columns
  import random 
  n=random.choice(range(1,features.size))
  feature_list=random.choices(features,k=n)
  feature_list
  frames = [X_train,y_train]
  data_train = pd.concat(frames,axis=1)
  n_sample=random.choice(range(int(data_train.count()[0]*0.7),data_train.count()[0]))
  data_train_sample=data_train.sample(n_sample,replace=bool)
  X_multi=data_train_sample[features]
  y_train_multi=[]
  for i in data_train_sample['class']:
      if i==1:
          y_train_multi.append([1,0])
      else:
          y_train_multi.append([0,1])         
  from sklearn import tree
  n_depth=random.choice(range(1,5))
  clf = tree.DecisionTreeClassifier(splitter='random',max_features='sqrt',max_depth=1)
  clf = clf.fit(X_multi, y_train_multi)
  #print(tree.export_text(clf))
  predictions=clf.predict(X_test[features])  
  H.append(predictions)

In [71]:
y_test_multi_true=[]
y_test_multi_false=[]
for i in y_test['class']:
      if i==1:
          y_test_multi_true.append([1,0])
      else:
          y_test_multi_true.append([0,1])
for i in y_test['class']:
      if i==1:
          y_test_multi_false.append([0,1])
      else:
          y_test_multi_false.append([1,0])          

In [72]:
n = np.array(H).shape[0]
w=cp.Variable(shape=(n,1),nonneg=True)
print(w)
#w = np.array([[0.25], [0.25], [0.25], [0.25]])
#print(np.array(H).shape)
majority = 0
print(n)
for i in range(n):
  majority = majority+w[i]*H[i]
#majority=w[0]*H[0]+w[1]*H[1]+w[2]*H[2]+w[3]*H[3]


var1253775
4


In [73]:
majority.shape

(171, 2)

In [74]:
np.array(y_test_multi_true).shape

(171, 2)

In [75]:
#mu_1=0
#mu_2=0
#for i,j,k in zip(majority,y_test_multi_true,y_test_multi_false):
#  mu_1=mu_1+(i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1]))
#  mu_2=mu_2+((i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1])))**2
margin = cp.sum(cp.multiply(majority, cp.atoms.affine.transpose.transpose(y_test_multi_true)), axis=1)
#margin = margin - cp.sum(cp.multiply(majority, cp.atoms.affine.transpose.transpose(y_test_multi_false)), axis=1)

mu_1 = cp.sum(margin)/majority.shape[0]
mu_2 = cp.sum(margin**2)/majority.shape[0]

In [80]:
u1=np.linspace(0.0,1.0,20)
#for t in t1:
for mu in u1:
  
  u=1
  l=0
  
  while u-l>0.001:
    t=(u+l)/2
    fn=mu**2-(1-t)*mu_2
    #fn=(u**2)/(mu_2)
    #problem = cp.Problem(cp.Maximize(fn),[mu_1==u,cp.sum(w)==1,w>=0])
  
  
    problem = cp.Problem(cp.Maximize(fn),[mu_1==mu,cp.sum(w)==1,w>=0])
    #try:
    #assert problem.is_dqcp()
    #op=problem.solve(verbose=True)
    try:
      op=problem.solve(qcp=True)
      u=t
      if w[0].value >0:
      #op=problem.solve(qcp=True, verbose=True)
        print(w.value,'u',mu,'t',t)
    except:
      print("-_('')_-")  
      l=t
      #print('fn_value ',op,' ','mu ',u,' ',
      #except:


-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
-_('')_-
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.5
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.25
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.125
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.0625
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.03125
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.015625
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.0078125
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.00390625
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.001953125
[[0.41332212]
 [0.28898002]
 [0.03381822]
 [0.26387963]] u 0.6842105263157894 t 0.000976562

if (4.26) is feasible, u := t; else l := t.
until u − l ≤ ǫ.

In [86]:
from sklearn import datasets
# import some data to play with
iris = datasets.load_iris()

In [87]:
#import data
X = iris.data
Y = iris.target
#X = X[0:5, :]
#Y = Y[0:5]
data = pd.DataFrame(X, columns=iris.feature_names)   # converting dataset into pandas dataframe for preprocessing
data['class'] = Y


In [95]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('class',axis=1),data.iloc[:,4:5], test_size=0.3, random_state=42)

In [96]:
n=4
H=[]
for i in range(0,n):
  features=data.drop('class',axis=1).columns
  import random 
  n=random.choice(range(1,features.size))
  feature_list=random.choices(features,k=n)
  feature_list
  frames = [X_train,y_train]
  data_train = pd.concat(frames,axis=1)
  n_sample=random.choice(range(int(data_train.count()[0]*0.7),data_train.count()[0]))
  data_train_sample=data_train.sample(n_sample,replace=bool)
  X_multi=data_train_sample[features]
  y_train_multi=[]
  for i in data_train_sample['class']:
      if i==0:
          y_train_multi.append([0,0,1])
      elif i==1:
          y_train_multi.append([0,1,0])
      else:
          y_train_multi.append([1,0,0])           
  from sklearn import tree
  n_depth=random.choice(range(1,5))
  clf = tree.DecisionTreeClassifier(splitter='random',max_features='sqrt',max_depth=1)
  clf = clf.fit(X_multi, y_train_multi)
  #print(tree.export_text(clf))
  predictions=clf.predict(X_test[features])  
  H.append(predictions)

In [101]:
y_test_multi_true=[]
y_test_multi_false=[]
for i in y_test['class']:
      if i==0:
          y_test_multi_true.append([0,0,1])
          y_test_multi_false.append([[0,1,0],[1,0,0]])
      elif i==1:
          y_test_multi_true.append([0,1,0])
          y_test_multi_false.append([[0,0,1],[1,0,0]])
      else:
          y_test_multi_true.append([1,0,0])
          y_test_multi_false.append([[0,1,0],[0,0,1]])       

In [108]:
n = np.array(H).shape[0]
w=cp.Variable(shape=(n,1),nonneg=True)
print(w)
#w = np.array([[0.25], [0.25], [0.25], [0.25]])
#print(np.array(H).shape)
majority = 0
print(n)
for i in range(n):
  majority = majority+w[i]*H[i]
#majority=w[0]*H[0]+w[1]*H[1]+w[2]*H[2]+w[3]*H[3]

var1365798
4


In [120]:
#mu_1=0
#mu_2=0
#for i,j,k in zip(majority,y_test_multi_true,y_test_multi_false):
#  mu_1=mu_1+(i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1]))
#  mu_2=mu_2+((i[0]*j[0])+(i[1]*j[1])-((i[0]*k[0])+(i[1]*k[1])))**2
margin1 = cp.sum(cp.multiply(majority, cp.atoms.affine.transpose.transpose(y_test_multi_true)), axis=1)
margin2 = cp.sum(cp.multiply(majority, (np.array(y_test_multi_false)[:,0])), axis=1)
margin3 = cp.sum(cp.multiply(majority, (np.array(y_test_multi_false)[:,1])), axis=1)

#margin = margin - cp.sum(cp.multiply(majority, cp.atoms.affine.transpose.transpose(y_test_multi_false)), axis=1)
mu_1 = cp.sum(margin1)/majority.shape[0]
mu_2 = cp.sum(margin1**2)/majority.shape[0]
#mu_1 = cp.sum(margin1-cp.maximum(margin2,margin3))/majority.shape[0]
#mu_2 = cp.sum((margin1-cp.maximum(margin2,margin3))**2)/majority.shape[0]
print(mu_1)

Sum(Sum((0.0 + Promote(var1365798[0, 0], (45, 3)) * [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]] + Promote(var1365798[1, 0], (45, 3)) * [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 

In [121]:
u1=np.linspace(0.0,1.0,20)
#for t in t1:
for mu in u1:
  
  u=1
  l=0
  
  while u-l>0.001:
    t=(u+l)/2
    fn=mu**2-(1-t)*mu_2
    #fn=(u**2)/(mu_2)
    #problem = cp.Problem(cp.Maximize(fn),[mu_1==u,cp.sum(w)==1,w>=0])
  
  
    problem = cp.Problem(cp.Maximize(fn),[mu_1==mu,cp.sum(w)==1,w>=0])
    #try:
    #assert problem.is_dqcp()
    #op=problem.solve(verbose=True)
    try:
      op=problem.solve(qcp=True)
      u=t
      if w[0].value >0:
      #op=problem.solve(qcp=True, verbose=True)
        print(w.value,'u',mu,'t',t)
    except:
      print("-_('')_-")  
      l=t
      #print('fn_value ',op,' ','mu ',u,' ',
      #except:


-_('')_-
-_('')_-
-_('')_-
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.5
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.25
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.125
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.0625
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.03125
[[5.79241400e-02]
 [1.99647163e-01]
 [7.42428697e-01]
 [4.53057299e-23]] u 0.2631578947368421 t 0.015625
[[5.79241400e-02]
 [1.99647163e-01]
 [7.42428697e-01]
 [1.13479419e-22]] u 0.2631578947368421 t 0.0078125
[[0.05792414]
 [0.19964716]
 [0.7424287 ]
 [0.        ]] u 0.2631578947368421 t 0.00390625
[[5.79241400e-02]
 [1.99647163e-01]
 [7.42428697e-01]
 [4.77787001e-23]] u 0.2631578947368421 t 0.001953125
[[5.79241400e-02]
 [1.99647163e-01]
 [7.42428697e-01]
 [1.76023708e-23]] u 0.2631578947368421 t 0.0009765625
[[0.19464863]
 [0.137900